In [12]:
import numpy as np
import re
from typing import Dict, Optional

rows      = 'ABCDEFGHI'
cols      = '123456789'
# squares: list comprehension that generates a list of strings representing all the cells in the Sudoku grid, such as 'A1', 'A2', ..., 'D4'.
squares   = [r + c for c in cols for r in rows] 

# all_boxes: list comprehension that generates a list of tuples representing the 9x9 boxes in the Sudoku grid. 
# Each tuple contains the cell labels belonging to a particular box.
all_boxes = [tuple(r + s for r in rs for s in cs) for rs in ('ABC','DEF', 'GHI') for cs in ('123','456', '789')] 

# all_units:  list containing tuples, each representing a group of cells (unit) in the Sudoku grid. These units include rows, columns, and boxes.
all_units = [tuple(r + C for r in rows for C in c) for c in cols] + [ tuple(R + c for R in r for c in cols) for r in rows] + all_boxes

# dictionary where keys are individual cells and values are tuples of units that contain that cell.
units     = {s: tuple(u for u in all_units if s in u) for s in squares}

# peers:  dictionary where keys are individual cells and values are sets of other cells (peers) 
# that are in the same row, column, or box as the key cell.
peers     = {s: set().union(*units[s]) - {s} for s in squares} 

In [13]:
all_boxes

[('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'),
 ('A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6'),
 ('A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9'),
 ('D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3'),
 ('D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'),
 ('D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9'),
 ('G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3'),
 ('G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6'),
 ('G7', 'G8', 'G9', 'H7', 'H8', 'H9', 'I7', 'I8', 'I9')]

In [14]:
all_units

[('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
 ('A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'),
 ('A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'),
 ('A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'),
 ('A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'),
 ('A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'),
 ('A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'),
 ('A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'),
 ('A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'I9'),
 ('A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'),
 ('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'),
 ('C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'),
 ('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'),
 ('E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'),
 ('F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'),
 ('G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'),
 ('H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'),
 ('I1', 'I2', 'I3', 'I4', 'I5',

In [15]:
units

{'A1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'),
  ('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3')),
 'B1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'),
  ('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3')),
 'C1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'),
  ('A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3')),
 'D1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'),
  ('D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3')),
 'E1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'),
  ('D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3')),
 'F1': (('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'),
  ('F1', 'F2', 'F3', 'F4', 'F5'

In [16]:
peers

{'A1': {'A2',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A8',
  'A9',
  'B1',
  'B2',
  'B3',
  'C1',
  'C2',
  'C3',
  'D1',
  'E1',
  'F1',
  'G1',
  'H1',
  'I1'},
 'B1': {'A1',
  'A2',
  'A3',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B8',
  'B9',
  'C1',
  'C2',
  'C3',
  'D1',
  'E1',
  'F1',
  'G1',
  'H1',
  'I1'},
 'C1': {'A1',
  'A2',
  'A3',
  'B1',
  'B2',
  'B3',
  'C2',
  'C3',
  'C4',
  'C5',
  'C6',
  'C7',
  'C8',
  'C9',
  'D1',
  'E1',
  'F1',
  'G1',
  'H1',
  'I1'},
 'D1': {'A1',
  'B1',
  'C1',
  'D2',
  'D3',
  'D4',
  'D5',
  'D6',
  'D7',
  'D8',
  'D9',
  'E1',
  'E2',
  'E3',
  'F1',
  'F2',
  'F3',
  'G1',
  'H1',
  'I1'},
 'E1': {'A1',
  'B1',
  'C1',
  'D1',
  'D2',
  'D3',
  'E2',
  'E3',
  'E4',
  'E5',
  'E6',
  'E7',
  'E8',
  'E9',
  'F1',
  'F2',
  'F3',
  'G1',
  'H1',
  'I1'},
 'F1': {'A1',
  'B1',
  'C1',
  'D1',
  'D2',
  'D3',
  'E1',
  'E2',
  'E3',
  'F2',
  'F3',
  'F4',
  'F5',
  'F6',
  'F7',
  'F8',
  'F9',
  'G1',
  'H1',
  'I1'}

In [17]:
def is_solution(solution, puzzle):
    """
    Check if the proposed solution to the puzzle is valid.

    Args:
    - solution: The proposed solution grid
    - puzzle: The original puzzle grid

    Returns:
    - True if the solution is valid, False otherwise
    """
    # Check if solution is not None, and all digits in each square of the solution are valid
    if solution is not None and all(solution[s] in puzzle[s] for s in squares):
        # Check if each unit in the solution contains all valid digits
        if all({solution[s] for s in unit} == {'1', '2', '3', '4', '5', '6', '7', '8', '9'} for unit in all_units):
            return True
    
    return False

In [18]:
def constrain(grid):
    "Propagate constraints on a copy of grid to yield a new constrained Grid."
    result = {s: '123456789' for s in squares}
    for s in grid:
        if len(grid[s]) == 1:
            fill(result, s,  grid[s])
    return result
    
def fill(grid, s, d):
    """
    Eliminate all the digits except d from grid[s]. e.g d = 3, inital grid[s] = 123456789, change grid[s] to 3

    Args:
    - grid: The Sudoku grid
    - s: The square to fill
    - d: The digit to fill

    Returns:
    - The modified grid if successful, otherwise None
    """
    # If grid[s] is already d or all other digits can be eliminated, return the grid
    if grid[s] == d or all(eliminate(grid, s, d2) for d2 in grid[s] if d2 != d):
        return grid
    else:
        return None
        
def eliminate(grid, s, d):
    """
    Eliminate d from grid[s]; implement the two constraint propagation strategies.
    
    Args:
    - grid: The Sudoku grid
    - s: The square to eliminate from
    - d: The digit to eliminate

    Returns:
    - The modified grid or None if the elimination is invalid
    """
    # If d is already eliminated, return the grid
    if d not in grid[s]:
        return grid  # Already eliminated
    
    # Eliminate d from the square s
    grid[s] = grid[s].replace(d, '')
    
    # If no legal digit left in square s, return None
    if not grid[s]:
        return None
    
    # If square s has only one possible digit left
    elif len(grid[s]) == 1:
        d2 = grid[s]
        # Eliminate d2 from each of the square's peers
        for s2 in peers[s]:
            if d2 in grid[s2]:
                if len(grid[s2]) == 1:
                    return None  # Can't eliminate d2 from some square
                if not eliminate(grid, s2, d2):
                    return None  # Can't eliminate d2 from some square
    
    # Check the constraint propagation strategies for each unit of square s
    for u in units[s]:
        dplaces = [s for s in u if d in grid[s]]
        # If no place in u for d or if a unit has only one possible square that can hold a digit
        if not dplaces or (len(dplaces) == 1 and not fill(grid, dplaces[0], d)):
            return None  # None: no place in u for d
    
    return grid

def find_next_empty_cell(grid):
    """
    Find the coordinates of the next empty cell in the Sudoku grid.
    
    Parameters:
        sudoku (list of lists): The Sudoku grid.
        
    Returns:
        tuple or None: The coordinates (row, col) of the next empty cell, or None if all cells are filled.
    """

    s = next((s for s in squares if len(grid[s])>1), None)

    return s

In [23]:
def parse(grid):
    assert grid.shape == (9, 9), "Ensure the shape of the sudoku is 9 by 9"
    result = {}
    for row, d in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']):
        for col in range(1, 10):
            key = f'{d}{col}' # A1, A2, A3, A4, ...
            value = grid[row][col-1]
            value = value if value != 0 else '123456789'
            result[key] = str(value)
    return result

def picture(grid):
    if grid is None: 
        return "None"
    
    def val(d):
        return '.' if d == '123456789' else d if len(d) == 1 else '{' + d + '}'
    
    maxwidth = max(len(val(grid[s])) for s in grid)
    dash1 = '-' * (maxwidth * 3 + 2)
    dash3 = '\n' + '+'.join(3 * [dash1])
    
    def cell(r, c):
        return val(grid[r + c]).center(maxwidth) + ('|'  if c in '36' else ' ')
    
    def line(r):
        return ''.join(cell(r, c) for c in cols) + (dash3 if r in 'CF' else '')
    
    return '\n'.join(map(line, rows))

In [27]:
sudoku = [[0, 0, 0, 0, 0, 0, 0, 2, 0],
          [0, 2, 0, 0, 0, 0, 5, 0, 0],
          [0, 0, 7, 0, 0, 3, 4, 0, 0],
          [2, 0, 0, 1, 0, 0, 3, 4, 0],
          [6, 4, 0, 0, 8, 0, 0, 5, 9],
          [0, 9, 5, 0, 0, 2, 0, 0, 1],
          [0, 0, 3, 4, 0, 0, 8, 0, 0],
          [0, 0, 9, 0, 0, 0, 0, 1, 0],
          [0, 1, 0, 0, 0, 0, 0, 0, 0],
         ]

sudoku = np.array(sudoku)

grid1 = parse(sudoku)
print(picture(grid1))

. . .|. . .|. 2 . 
. 2 .|. . .|5 . . 
. . 7|. . 3|4 . . 
-----+-----+-----
2 . .|1 . .|3 4 . 
6 4 .|. 8 .|. 5 9 
. 9 5|. . 2|. . 1 
-----+-----+-----
. . 3|4 . .|8 . . 
. . 9|. . .|. 1 . 
. 1 .|. . .|. . . 


In [28]:
def solve_puzzles(search_funct, puzzles):
    "Solve and verify each puzzle, and print puzzle and solution."
    # Iterate over each puzzle in the list of puzzles.
    for puzzle in puzzles:
        # Solve the puzzle using the provided search function and apply constraint propagation.
        solution = search_funct(constrain(puzzle))
        
        # Verify if the solution satisfies the constraints of a valid Sudoku solution.
        assert is_solution(solution, puzzle), "No solution"
        
        # Print the puzzle and its solution side by side.
        print_side_by_side('\nPuzzle:\n' + picture(puzzle), 
                           '\nSolution:\n' + picture(solution))

    
def print_side_by_side(left_text, right_text):
    """
    Print two strings side-by-side, line-by-line, each side `width` wide.

    Args:
        left_text (str): The text to print on the left side.
        right_text (str): The text to print on the right side.
    """
    left_lines = left_text.splitlines()
    right_lines = right_text.splitlines()
    width = 20

    for left_line, right_line in zip(left_lines, right_lines):
        left_padded = left_line.ljust(width)
        right_padded = right_line.ljust(width)
        print(left_padded, right_padded)

# Algorithms

In [29]:
from queue import PriorityQueue
from collections import deque

def a_star(grid):
    "A* Search with constraint propagation to find a solution."
    if grid is None: 
        return None
    
    frontier = PriorityQueue()
    frontier.put((0, grid))  # Initialize the frontier with the initial state and its evaluation value
    
    while not frontier.empty():
        _, current_grid = frontier.get()

        # Check if the grid is a solution
        if all(len(current_grid[s]) == 1 for s in squares):
            return current_grid

        # Find the next square with the empty cell
        s = find_next_empty_cell(current_grid)
        
        if s is None:  # No squares with multiple possibilities; the search has succeeded
            continue

        for d in current_grid[s]:
            new_grid = fill(current_grid.copy(), s, d)
            if new_grid:
                cost = calculate_cost(current_grid, new_grid, s)
                heuristic_value = heuristic(new_grid)  # Calculate heuristic value for the new state
                evaluation_value = cost + heuristic_value
                frontier.put((evaluation_value, new_grid))  # Add the new state to the frontier with its evaluation value

    return None

def calculate_cost(old_grid, new_grid, square):
    "Calculate the cost for transitioning from the old grid to the new grid."
    # In A* search, the cost is typically the cumulative cost from the start state to the current state
    # Here, we use the length of possibilities in the square as the cost
    return len(old_grid[square]) - 1
    
def heuristic(grid) -> int:
    "A simple heuristic function that counts the number of empty squares."
    return sum(1 for s in squares if len(grid[s]) > 1)

solve_puzzles(a_star,[grid1])

                                         
Puzzle:              Solution:           
. . .|. . .|. 2 .    9 3 4|5 6 8|1 2 7   
. 2 .|. . .|5 . .    8 2 6|7 1 4|5 9 3   
. . 7|. . 3|4 . .    1 5 7|9 2 3|4 6 8   
-----+-----+-----    -----+-----+-----   
2 . .|1 . .|3 4 .    2 7 8|1 5 9|3 4 6   
6 4 .|. 8 .|. 5 9    6 4 1|3 8 7|2 5 9   
. 9 5|. . 2|. . 1    3 9 5|6 4 2|7 8 1   
-----+-----+-----    -----+-----+-----   
. . 3|4 . .|8 . .    5 6 3|4 9 1|8 7 2   
. . 9|. . .|. 1 .    7 8 9|2 3 5|6 1 4   
. 1 .|. . .|. . .    4 1 2|8 7 6|9 3 5   
